<a href="https://colab.research.google.com/github/rmcgrath050/Learning-Journal/blob/main/python_hackathon_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Python Data Hackathon (Single-Notebook Edition)

**Objectives**
- Import one CSV and one log file into pandas
- Clean (missing values, deduplicate)
- Normalise (split into tidy tables with sensible keys)
- Save outputs as CSVs
- Light EDA (3 simple plots + 3 insights)
- Refactor to run end-to-end in one go with functions + logging

> **Tip:** Keep this notebook runnable top-to-bottom. Use the final **Run Pipeline** cell to test end-to-end.


## 🗓️ Day Plan (2 × 3 hours, incl. 20‑min break per block)

**Block 1 (~3h)**
1. Briefing & dataset walkthrough (20–25m)
2. Ingest CSV + log → Clean (nulls, duplicates) (65–75m)
3. **Break (20m)**
4. Normalise → Save tidy tables (40–45m)
5. Light EDA → 3 plots + 3 insights (25–30m)

**Block 2 (~3h)**
1. Quick recap + quality bar (10–15m)
2. Refactor to functions + logging + one-click run (80–90m)
3. **Break (20m)**
4. Polish: comments/docstrings, README notes, acceptance checks (35–45m)
5. Demos/peer review (10–20m)


## ⚙️ Setup: paths, logging, imports
Use this once at the start. Re-run if you restart the kernel.


In [ ]:
from pathlib import Path
import logging
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime

# ---- Parameters (edit as needed) ----
#path functions create folders
RAW_DIR = Path("data/raw") # data folder
PROC_DIR = Path("data/processed") # processed sub folder in data

LOG_DIR = Path("logs") # logs folder.
LOG_FILE = LOG_DIR / "hackathon.log" #hackathon sub folder

REPORTS_DIR = Path("reports") # reports folder
FIGURES_DIR = REPORTS_DIR / "figures" # figures subfolder


# Expected files (place in data/raw)
CSV_FILE = RAW_DIR / "orders.csv"           # <- change as needed
LOG_FILE_PATH = RAW_DIR / "access.log"      # <- change as needed

# ---- Create folders (can also use bash: !mkdir -p data/raw data/processed logs reports/figures) ----
for p in [RAW_DIR, PROC_DIR, LOG_DIR, REPORTS_DIR, FIGURES_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# ---- Logging config (file + console) ----
logger = logging.getLogger("hackathon")
logger.setLevel(logging.INFO)
logger.handlers.clear()

fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")

fh = logging.FileHandler(LOG_FILE, encoding='utf-8')
fh.setFormatter(fmt)
fh.setLevel(logging.INFO)
logger.addHandler(fh)

ch = logging.StreamHandler(sys.stdout)
ch.setFormatter(fmt)
ch.setLevel(logging.INFO)
logger.addHandler(ch)

logger.info("Setup complete. Folders ensured. Ready to start.")


2026-01-24 22:25:36,979 | INFO | Setup complete. Folders ensured. Ready to start.


INFO:hackathon:Setup complete. Folders ensured. Ready to start.


## 🧰 Utilities (acceptance checks & helpers)
You can extend these if helpful.


In [ ]:
def log_shape(df, name: str):
    logger.info(f"{name}: {df.shape[0]} rows × {df.shape[1]} cols")

def log_nulls(df, name: str):
    null_pct = (df.isna().mean() * 100).round(2)
    tops = null_pct.sort_values(ascending=False).head(5).to_dict()
    logger.info(f"{name}: top null% -> {tops}")

def assert_unique(df, cols, name="frame"):
    if df.duplicated(subset=cols).any():
        dups = df[df.duplicated(subset=cols, keep=False)]
        logger.error(f"{name}: duplicates found on {cols}")
        raise AssertionError(f"{name}: duplicates on {cols}")
    logger.info(f"{name}: unique on {cols}")

def assert_no_orphans(child, parent, fk, pk, child_name="child", parent_name="parent"):
    missing = ~child[fk].isin(parent[pk])
    if missing.any():
        n = int(missing.sum())
        logger.error(f"{child_name}: {n} orphan rows where {fk} not in {parent_name}.{pk}")
        raise AssertionError(f"{child_name}: orphan FKs: {n}")
    logger.info(f"{child_name}: no orphan {fk} against {parent_name}.{pk}")

def save_table(df, name):
    out = PROC_DIR / f"{name}.csv"
    df.to_csv(out, index=False)
    logger.info(f"Saved {name} -> {out}")
    return out

def save_fig_current(name):
    out = FIGURES_DIR / f"{name}.png"
    plt.savefig(out, bbox_inches="tight", dpi=150)
    logger.info(f"Saved figure -> {out}")
    return out


## 🔽 Block 1: Ingest → Clean
**Goal:** Read one CSV and one log file into pandas; handle nulls & duplicates with simple, defensible rules; log key metrics.


In [ ]:
# --- 1.1 Load CSV ---
def load_csv(path: Path) -> pd.DataFrame:
    logger.info(f"Loading CSV: {path}")
    df = pd.read_csv(path)
    log_shape(df, "raw_csv")
    log_nulls(df, "raw_csv")
    return df

try:
    df_csv = load_csv(CSV_FILE)
except FileNotFoundError:
    logger.error(f"CSV not found at {CSV_FILE}. Place your CSV in data/raw and re-run.")
    df_csv = pd.DataFrame()


In [ ]:
# --- 1.2 Load & parse log ---
LOG_PATTERN = re.compile(
    r"^(?P<ip>\S+)\s+-\s+-\s+\[(?P<ts>[^\]]+)\]\s+\"(?P<method>\S+)\s+(?P<path>\S+)\s+(?P<proto>\S+)\"\s+(?P<status>\d{3})\s+(?P<size>\S+)"  # Apache-like
)

def parse_log_line(line: str):
    m = LOG_PATTERN.search(line)
    if not m:
        return None
    d = m.groupdict()
    # quick cast
    d["status"] = int(d["status"]) if d.get("status") else None
    d["size"] = int(d["size"]) if d.get("size") and d["size"].isdigit() else None
    # timestamp parse (example format: 10/Oct/2000:13:55:36 -0700)
    try:
        d["ts"] = datetime.strptime(d["ts"], "%d/%b/%Y:%H:%M:%S %z")
    except Exception:
        pass
    return d

def load_log(path: Path) -> pd.DataFrame:
    logger.info(f"Loading log: {path}")
    rows = []
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                d = parse_log_line(line)
                if d:
                    rows.append(d)
        df = pd.DataFrame(rows)
        log_shape(df, "raw_log")
        log_nulls(df, "raw_log")
        return df
    except FileNotFoundError:
        logger.error(f"Log not found at {path}. Place your log in data/raw and re-run.")
        return pd.DataFrame()

df_log = load_log(LOG_FILE_PATH)


### 🧽 Cleaning functions
Adjust rules as appropriate for your dataset. Be explicit in decisions.


In [ ]:
def clean_frame(df: pd.DataFrame, *, key_cols=None, fill_rules=None, drop_na_thresh=None, name="frame") -> pd.DataFrame:
    """Generic cleaner: dedupe on key_cols; fill using fill_rules; optional dropna threshold."""
    if df.empty:
        logger.warning(f"{name}: empty frame passed to clean_frame")
        return df.copy()
    before = len(df)
    if key_cols:
        df = df.drop_duplicates(subset=key_cols)
    else:
        df = df.drop_duplicates()
    d_removed = before - len(df)
    if d_removed:
        logger.info(f"{name}: removed {d_removed} duplicates")

    # Fill rules: {'col': value or function}
    fill_rules = fill_rules or {}
    for col, val in fill_rules.items():
        if callable(val):
            df[col] = df[col].apply(lambda x: val(x))
        else:
            df[col] = df[col].fillna(val)

    if drop_na_thresh is not None:
        # Drop rows with too many NAs
        df = df.dropna(thresh=drop_na_thresh)

    log_shape(df, f"clean_{name}")
    log_nulls(df, f"clean_{name}")
    return df

# Example usage (customise):
df_clean = clean_frame(
    df_csv,
    key_cols=["order_id"] if "order_id" in df_csv.columns else None,
    fill_rules={},
    drop_na_thresh=None,
    name="csv"
)


## 🔀 Normalise → tidy tables
Split into entities (e.g., users, products, orders, order_items). Adjust to your schema. Validate with unique PKs and non-orphan FKs.


In [ ]:
def to_tidy_tables(df: pd.DataFrame):
    """Example normalisation for an orders-like CSV with columns such as:
    order_id, user_id, user_name, product_id, product_name, quantity, unit_price, order_ts
    Adjust for your dataset. Returns dict of DataFrames.
    """
    tables = {}
    if df.empty:
        return tables

    cols = df.columns.str.lower().tolist()

    # Users
    user_cols = [c for c in cols if c.startswith("user_") or c == "user_id"]
    if "user_id" in cols:
        users = (
            df[[c for c in df.columns if c.lower() in set(user_cols)]]
            .drop_duplicates()
            .rename(columns=lambda c: c.lower())
        )
        assert_unique(users, ["user_id"], name="users")
        tables["users"] = users

    # Products
    prod_cols = [c for c in cols if c.startswith("product_") or c == "product_id"]
    if "product_id" in cols:
        products = (
            df[[c for c in df.columns if c.lower() in set(prod_cols)]]
            .drop_duplicates()
            .rename(columns=lambda c: c.lower())
        )
        assert_unique(products, ["product_id"], name="products")
        tables["products"] = products

    # Orders
    if "order_id" in cols:
        order_cols = [c for c in df.columns if c.lower() in {"order_id", "user_id", "order_ts", "order_date"}]
        orders = df[order_cols].drop_duplicates().rename(columns=lambda c: c.lower())
        assert_unique(orders, ["order_id"], name="orders")
        if "users" in tables:
            assert_no_orphans(orders, tables["users"], "user_id", "user_id", child_name="orders", parent_name="users")
        tables["orders"] = orders

    # Order items
    if set(["order_id", "product_id"]).issubset(set(cols)):
        oi_cols = [c for c in df.columns if c.lower() in {"order_id", "product_id", "quantity", "unit_price"}]
        order_items = df[oi_cols].rename(columns=lambda c: c.lower())
        # create a surrogate key if not present
        order_items = order_items.copy()
        order_items.insert(0, "order_item_id", range(1, len(order_items) + 1))
        assert_no_orphans(order_items, tables.get("orders", df[["order_id"]].drop_duplicates().rename(columns=lambda c: c.lower())),
                         "order_id", "order_id", child_name="order_items", parent_name="orders")
        if "products" in tables:
            assert_no_orphans(order_items, tables["products"], "product_id", "product_id", child_name="order_items", parent_name="products")
        tables["order_items"] = order_items

    return tables

tables = to_tidy_tables(df_clean)
for name, tdf in tables.items():
    save_table(tdf, name)


## 📈 Light EDA (save figures + write 3 insights)
Create at least 3 basic plots (histogram, bar/column, line). Save images under `reports/figures/` and summarise insights in `reports/EDA_summary.md`.


In [ ]:
# Example EDA skeleton — adjust columns to your dataset
EDA_NOTES = REPORTS_DIR / "EDA_summary.md"

def run_eda(df: pd.DataFrame):
    if df.empty:
        logger.warning("EDA skipped: empty frame")
        return

    # Histogram example
    if "unit_price" in df.columns:
        df["unit_price"].dropna().plot(kind="hist", bins=20)
        plt.title("Distribution of unit_price")
        plt.xlabel("unit_price")
        save_fig_current("hist_unit_price")
        plt.close()

    # Bar/column example
    if set(["product_id", "quantity"]).issubset(df.columns):
        (df.groupby("product_id")["quantity"].sum().sort_values().tail(10)).plot(kind="bar")
        plt.title("Top 10 products by quantity")
        plt.xlabel("product_id")
        plt.ylabel("total quantity")
        save_fig_current("bar_top_products")
        plt.close()

    # Line example
    for time_col in ["order_ts", "order_date", "ts"]:
        if time_col in df.columns:
            ts = df.dropna(subset=[time_col]).copy()
            ts[time_col] = pd.to_datetime(ts[time_col], errors="coerce")
            g = ts.groupby(pd.Grouper(key=time_col, freq="D")).size()
            if len(g) > 0:
                g.plot(kind="line")
                plt.title(f"Daily counts by {time_col}")
                plt.xlabel("date")
                plt.ylabel("count")
                save_fig_current(f"line_daily_counts_{time_col}")
                plt.close()
                break

    # Write 3 insights (edit these to be data-specific)
    EDA_NOTES.write_text(
        """
# EDA Summary (edit me)

1) Describe a pattern you see in the histogram (e.g., skew/right-tail in unit_price).
2) Describe the top categories/products and a possible reason.
3) Describe the temporal trend (e.g., weekly cycles or anomalies).
        """.strip(),
        encoding="utf-8"
    )
    logger.info(f"Wrote EDA notes -> {EDA_NOTES}")


## 🧩 Block 2: Refactor → one-click run
Wrap steps into functions and a simple orchestrator. Use the **Run Pipeline** cell below to execute end-to-end.


In [ ]:
def run_pipeline():
    logger.info("===== PIPELINE START =====")
    # Ingest
    csv_df = load_csv(CSV_FILE)
    log_df = load_log(LOG_FILE_PATH)

    # Clean (customise rules/keys as needed)
    clean_df = clean_frame(
        csv_df,
        key_cols=["order_id"] if "order_id" in csv_df.columns else None,
        fill_rules={},
        name="csv"
    )

    # Normalise
    tidy = to_tidy_tables(clean_df)
    for name, tdf in tidy.items():
        save_table(tdf, name)

    # EDA
    run_eda(clean_df)

    logger.info("===== PIPELINE COMPLETE =====")
    return {k: v.shape for k, v in tidy.items()}


In [ ]:
# ▶️ Run Pipeline (edit file names/columns earlier as needed, then run this cell)
shapes = run_pipeline()
shapes


## ✅ Definition of Done (DoD)
- Notebook runs top-to-bottom without error.
- `data/processed/` contains normalised tables (CSV).
- `reports/figures/` contains 3 saved plots.
- `logs/hackathon.log` records steps + metrics (rows in/out, null %, duplicates removed).
- `reports/EDA_summary.md` has 3 concise insights.
- Code has functions, clear comments, and meaningful names.


## 🤝 Peer Review Checklist (10–15 min)
- Can I run your notebook without touching cell order?
- Are assumptions/rules written down (e.g., how nulls were handled)?
- Are keys/relationships sensible? Any orphan checks?
- Do the plots have titles/labels and are they saved?
- Is the log file informative?


## 🌟 Stretch Goals
- Add simple unit tests (e.g., assert dedupe works) in a separate test cell.
- Support config via a small dictionary or JSON file read.
- Add `argparse`-style parameters using `papermill` tags (optional) or a top **Parameters** cell.
- Add a data dictionary under `reports/`.
